# 8.5 안전 운전자 예측 경진대회 성능 개선 II : XGBoost 모델

In [5]:
import pandas as pd

# 데이터 경로
data_path = 'data/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 8.5.1 피처 엔지니어링

### 데이터 합치기

In [6]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃값 제거

all_features = all_data.columns # 전체 피처

### 명목형 피처 원-핫 인코딩

In [7]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처
cat_features = [feature for feature in all_features if 'cat' in feature]

# 원-핫 인코딩 적용
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

### 파생 피처 추가

In [8]:
# '데이터 하나당 결측값 개수'를 파생 피처로 추가
all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [9]:
# 명목형 피처, calc 분류 피처를 제외한 피처
remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and 'calc' not in feature)] 
# num_missing을 remaining_features에 추가
remaining_features.append('num_missing')

In [10]:
# 분류가 ind인 피처
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) + '_'

In [11]:
all_data['mix_ind']

0          2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0_
1           1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1_
2          5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0_
3           0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_
4           0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0_
                           ...                  
1488023     0_1_6_0_0_0_1_0_0_0_0_0_0_0_2_0_0_1_
1488024    5_3_5_1_0_0_0_1_0_0_0_0_0_0_11_1_0_0_
1488025     0_1_5_0_0_1_0_0_0_0_0_0_0_0_5_0_0_1_
1488026    6_1_5_1_0_0_0_0_1_0_0_0_0_0_13_1_0_0_
1488027    7_1_4_1_0_0_0_0_1_0_0_0_0_0_12_1_0_0_
Name: mix_ind, Length: 1488028, dtype: object

In [12]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: 
                                                           val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')

In [13]:
cat_count_features

['ps_ind_02_cat_count',
 'ps_ind_04_cat_count',
 'ps_ind_05_cat_count',
 'ps_car_01_cat_count',
 'ps_car_02_cat_count',
 'ps_car_03_cat_count',
 'ps_car_04_cat_count',
 'ps_car_05_cat_count',
 'ps_car_06_cat_count',
 'ps_car_07_cat_count',
 'ps_car_08_cat_count',
 'ps_car_09_cat_count',
 'ps_car_10_cat_count',
 'ps_car_11_cat_count',
 'mix_ind_count']

### 필요 없는 피처 제거

In [14]:
from scipy import sparse

# 필요 없는 피처들
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# remaining_features, cat_count_features에서 drop_features를 제거한 데이터
all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

### 데이터 나누기

In [15]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

### 정규화 지니계수 계산 함수

In [16]:
import numpy as np

def eval_gini(y_true, y_pred):
    # 실제값과 예측값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]                      # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)       # 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true)       # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [17]:
# XGBoost용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)

## 8.5.2 하이퍼파라미터 최적화

### 데이터셋 준비

In [14]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)
# 베이지안 최적화용 데이터셋
bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

### 하이퍼파라미터 범위 설정

In [15]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {'max_depth': (4, 8),
                'subsample': (0.6, 0.9),
                'colsample_bytree': (0.7, 1.0),
                'min_child_weight': (5, 7),
                'gamma': (8, 11),
                'reg_alpha': (7, 9),
                'reg_lambda': (1.1, 1.5),
                'scale_pos_weight': (1.4, 1.6)}

# 값이 고정된 하이퍼파라미터
fixed_params = {'objective': 'binary:logistic',
                'learning_rate': 0.02,
                'random_state': 1991}

### (베이지안 최적화용) 평가지표 계산 함수 작성

In [16]:
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight,
                 reg_alpha, gamma, reg_lambda, scale_pos_weight):
    '''최적화하려는 평가지표(지니계수) 계산 함수'''
    # 베이지안 최적화를 수행할 하이퍼파라미터
    params = {'max_depth': int(round(max_depth)),
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'min_child_weight': min_child_weight,
              'gamma': gamma,
              'reg_alpha':reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight': scale_pos_weight}
    # 값이 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼파라미터 :', params)    
        
    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=params, 
                          dtrain=bayes_dtrain,
                          num_boost_round=2000,
                          evals=[(bayes_dvalid, 'bayes_dvalid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=False)
                           
    best_iter = xgb_model.best_iteration # 최적 반복 횟수
    # 검증 데이터로 예측 수행
    preds = xgb_model.predict(bayes_dvalid, 
                              iteration_range=(0, best_iter))
    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수 : {gini_score}\n')
    
    return gini_score

### 최적화 수행

In [17]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, 
                                 pbounds=param_bounds, 
                                 random_state=0)

# 베이지안 최적화 수행
optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
하이퍼파라미터 : {'max_depth': 6, 'subsample': 0.867531900234624, 'colsample_bytree': 0.8646440511781974, 'min_child_weight': 6.0897663659937935, 'gamma': 10.14556809911726, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\Users\Hi\anaconda3\envs\tf_python_310\lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.2770918995329271

| 1         | 0.2771    | 0.8646    | 10.15     | 6.411     | 6.09      | 7.847     | 1.358     | 1.488     | 0.8675    |
하이퍼파라미터 : {'max_depth': 7, 'subsample': 0.6261387899104622, 'colsample_bytree': 0.9890988281503088, 'min_child_weight': 6.0577898395058085, 'gamma': 9.150324556477333, 'reg_alpha': 8.136089122187865, 'reg_lambda': 1.4702386553170643, 'scale_pos_weight': 1.4142072116395774, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}
지니계수 : 0.2785348596472599

| 2         | 0.2785    | 0.9891    | 9.15      | 7.167     | 6.058     | 8.136     | 1.47      | 1.414     | 0.6261    |
하이퍼파라미터 : {'max_depth': 7, 'subsample': 0.8341587528859367, 'colsample_bytree': 0.7060655192320977, 'min_child_weight': 6.7400242964936385, 'gamma': 10.497859536643814, 'reg_alpha': 8.957236684465528, 'reg_lambda': 1.4196634256866894, 'scale_pos_weight': 1.4922958724505864, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}

c:\Users\Hi\anaconda3\envs\tf_python_310\lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.27763413318913566

| 4         | 0.2776    | 0.8843    | 10.45     | 6.838     | 6.494     | 8.552     | 1.381     | 1.423     | 0.7002    |
하이퍼파라미터 : {'max_depth': 7, 'subsample': 0.8535233675350644, 'colsample_bytree': 0.92975858050776, 'min_child_weight': 6.249564429359247, 'gamma': 9.95563546750357, 'reg_alpha': 8.411512219837842, 'reg_lambda': 1.424460008293778, 'scale_pos_weight': 1.5416807226581535, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


...

지니계수 : 0.2810739973185967

| 7         | 0.2811    | 0.8818    | 9.013     | 6.927     | 6.992     | 7.641     | 1.356     | 1.449     | 0.6931    |
하이퍼파라미터 : {'max_depth': 7, 'subsample': 0.6178380586143949, 'colsample_bytree': 0.8002721767872601, 'min_child_weight': 7.0, 'gamma': 8.482019636146331, 'reg_alpha': 7.600258088655923, 'reg_lambda': 1.2688863424536194, 'scale_pos_weight': 1.4659290254137904, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\Users\Hi\anaconda3\envs\tf_python_310\lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.28069036550059084

| 8         | 0.2807    | 0.8003    | 8.482     | 6.745     | 7.0       | 7.6       | 1.269     | 1.466     | 0.6178    |
하이퍼파라미터 : {'max_depth': 7, 'subsample': 0.7102431062533778, 'colsample_bytree': 1.0, 'min_child_weight': 7.0, 'gamma': 8.75239267900182, 'reg_alpha': 7.0, 'reg_lambda': 1.282397381262159, 'scale_pos_weight': 1.4538566545153988, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\Users\Hi\anaconda3\envs\tf_python_310\lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.2811927414631206

| 9         | 0.2812    | 1.0       | 8.752     | 7.405     | 7.0       | 7.0       | 1.282     | 1.454     | 0.7102    |


### 결과 확인

In [18]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 1.0,
 'gamma': 8.75239267900182,
 'max_depth': 7.404752040081974,
 'min_child_weight': 7.0,
 'reg_alpha': 7.0,
 'reg_lambda': 1.282397381262159,
 'scale_pos_weight': 1.4538566545153988,
 'subsample': 0.7102431062533778}

In [18]:
# 정수형 하이퍼파라미터 변환
max_params['max_depth'] = int(round(max_params['max_depth']))

# 값이 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)
max_params

NameError: name 'fixed_params' is not defined

## 8.5.3 모델 훈련 및 성능 검증

In [19]:
import numpy as np
import xgboost as xgb

max_params = {'colsample_bytree': 1.0,
 'gamma': 8.75239267900182,
 'max_depth': 7,
 'min_child_weight': 7.0,
 'reg_alpha': 7.0,
 'reg_lambda': 1.282397381262159,
 'scale_pos_weight': 1.4538566545153988,
 'subsample': 0.7102431062533778,
 'objective': 'binary:logistic',
 'learning_rate': 0.02,
 'random_state': 1991}

In [22]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 
valid_pred_prob = [[] for _ in range(5)]
# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # XGBoost 전용 데이터셋 생성 
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    dtest = xgb.DMatrix(X_test)
    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=max_params, 
                          dtrain=dtrain,
                          num_boost_round=2000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=100)

    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장
    best_iter = xgb_model.best_iteration
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter))/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, 
                                                  iteration_range=(0, best_iter))
    valid_pred_prob[idx] = xgb_model.predict(dvalid, iteration_range=(0, best_iter))

    # 검증 데이터 예측 확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


/Users/admin/anaconda3/envs/tf_env/lib/python3.11/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	valid-logloss:0.21788	valid-gini:0.20535
[100]	valid-logloss:0.16040	valid-gini:0.27362
[200]	valid-logloss:0.15516	valid-gini:0.28659
[300]	valid-logloss:0.15444	valid-gini:0.29124
[400]	valid-logloss:0.15433	valid-gini:0.29274
[500]	valid-logloss:0.15429	valid-gini:0.29312
[600]	valid-logloss:0.15427	valid-gini:0.29317
[700]	valid-logloss:0.15427	valid-gini:0.29352
[800]	valid-logloss:0.15426	valid-gini:0.29378
[900]	valid-logloss:0.15428	valid-gini:0.29386
[1000]	valid-logloss:0.15425	valid-gini:0.29397
[1100]	valid-logloss:0.15424	valid-gini:0.29403
[1200]	valid-logloss:0.15425	valid-gini:0.29408
[1300]	valid-logloss:0.15425	valid-gini:0.29407
[1400]	valid-logloss:0.15423	valid-gini:0.29422
[1500]	valid-logloss:0.15423	valid-gini:0.29433
[1600]	valid-logloss:0.15423	valid-gini:0.29438
[1700]	valid-logloss:0.15424	valid-gini:0.29445
[1800]	valid-logloss:0.15423	valid-gini:0.29451
[1900]	valid-logloss:0.15422	valid-gini:0.29452
[1999]	valid-logloss:0.15424	valid-gini:0.29465
폴드 1

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba_prob=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    auc = 0
    if pred_proba_prob:
        auc = roc_auc_score(y_test, pred_proba_prob)
    print("오차 행렬")
    print(confusion)
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")
    return confusion, accuracy, precision, recall, f1, auc
    
def test_model(threshold:float=0.5):
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)
    accuracy, precision, recall, f1 = 0, 0, 0, 0
    for idx, (_, valid_idx) in enumerate(folds.split(X, y)):
        _, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

        valid_pred = np.where(valid_pred_prob[idx]>threshold, 1, 0)
        _, valid_accuracy, valid_precision, valid_recall, valid_f1, _ = get_clf_eval(y_test=y_valid, pred=valid_pred)
        accuracy+=valid_accuracy/folds.n_splits
        precision+=valid_precision/folds.n_splits
        recall+=valid_recall/folds.n_splits
        f1+=valid_f1/folds.n_splits
        ###        
    print(f"정확도: {accuracy}, 정밀도: {precision}, 재현율: {recall}, F1: {f1}")
    return accuracy, precision, recall, f1


In [24]:
accuracy, precision, recall, f1 = 0, 0, 0, 0
best_threshold = 0
for threshold in np.arange(0, 0.3, 0.001):
    valid_accuracy, valid_precision, valid_recall, valid_f1 = test_model(threshold=threshold)
    if valid_f1>f1:
        f1, best_threshold = valid_f1, threshold
print('#'*80)
print(f"최적 threshold: {best_threshold}")
print(test_model(threshold=best_threshold))

오차 행렬
[[     0 114704]
 [     0   4339]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, F1: 0.0703, AUC: 0.0000
오차 행렬
[[     0 114704]
 [     0   4339]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, F1: 0.0703, AUC: 0.0000
오차 행렬
[[     0 114704]
 [     0   4338]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, F1: 0.0703, AUC: 0.0000
오차 행렬
[[     0 114703]
 [     0   4339]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, F1: 0.0703, AUC: 0.0000
오차 행렬
[[     0 114703]
 [     0   4339]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, F1: 0.0703, AUC: 0.0000
정확도: 0.036447517854154926, 정밀도: 0.036447517854154926, 재현율: 1.0, F1: 0.07033162261696932
오차 행렬
[[     0 114704]
 [     0   4339]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, F1: 0.0703, AUC: 0.0000
오차 행렬
[[     0 114704]
 [     0   4339]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, F1: 0.0703, AUC: 0.0000
오차 행렬
[[     0 114704]
 [     0   4338]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, F1: 0.0703, AUC: 0.0000
오차 행렬
[[     0 114703]
 [     0   4339]]
정확도: 0.0364, 정밀도: 0.0364, 재현율: 1.0000, 

In [21]:
print('OOF 검증 데이터 지니계수 :', eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수 : 0.2853918381420083


## 8.5.4 예측 및 결과 제출

In [22]:
submission['target'] = oof_test_preds
submission.to_csv('output/xgb_submission.csv')